In [10]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
import scipy
import scipy.stats as stats
import os

In [11]:
from utils.variables import pack, unpack
from utils.display import table, tabulate
from utils.files import download_sheet, save, load
from utils.hash import digest

from game_model import game
from rp_model import compute_rp, make_precomputed_columns
from initial_guess import make_initial_guess


In [12]:
# stuff for display

from IPython.lib.pretty import pretty, pprint

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 120)

In [13]:
# Load previous data

data = pd.read_pickle('./data/rp-data.pickle')
data.describe()
data.head()

,Level,RP,MS lvl,ModelRP,Difference,Freq1,FreqL,Inv,Berry1,BerryL,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,BerryD,IngD,Dupes,Amnt,Ing2P,Help skill bonus,RP Multiplier
count,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.0,5735.000000,5735.000000,5735.000000,5735.00000
mean,10.054577,585.904446,1.445336,584.709459,0.020009,3970.305144,3826.666908,12.123627,28.033653,37.155013,108.791805,1.010856,0.988570,0.994532,1.001883,1.005510,0.207906,31.745737,54.023540,147.813601,1.0,0.047602,1.873583,0.988221,1.02694
std,6.839047,371.356752,0.961285,369.419134,0.941834,947.560612,1011.749091,3.356021,2.614471,7.556406,15.702953,0.299620,0.059088,0.045185,0.118457,0.118344,0.044695,35.855732,26.374069,58.382610,0.0,0.420109,16.313341,0.036752,0.06850
min,1.000000,245.000000,1.000000,-1.000000,-13.710000,2200.000000,1489.118400,7.000000,24.000000,24.000000,90.000000,0.520000,0.890000,0.920000,0.800000,0.800000,0.000000,0.000000,25.000000,90.000000,1.0,0.000000,0.000000,0.790000,1.00000
25%,5.000000,352.000000,1.000000,351.455000,-0.280000,3300.000000,2992.680000,10.000000,26.000000,32.000000,98.000000,0.800000,1.000000,1.000000,1.000000,1.000000,0.180000,12.830000,35.000000,101.000000,1.0,0.000000,0.000000,1.000000,1.00000
50%,9.000000,448.000000,1.000000,447.830000,-0.020000,4000.000000,3819.200000,11.000000,28.000000,36.000000,103.000000,0.940000,1.000000,1.000000,1.000000,1.000000,0.200000,19.240000,44.000000,121.000000,1.0,0.000000,0.000000,1.000000,1.00000
75%,13.000000,692.000000,1.000000,690.490000,0.270000,4500.000000,4455.000000,14.000000,30.000000,41.000000,115.000000,1.200000,1.000000,1.000000,1.000000,1.000000,0.235000,43.125000,66.000000,202.000000,1.0,0.000000,0.000000,1.000000,1.00000
max,42.000000,3440.000000,6.000000,3440.330000,8.910000,6300.000000,6916.140000,23.000000,35.000000,74.000000,151.000000,2.410000,1.120000,1.080000,1.200000,1.200000,0.420000,413.800000,213.000000,511.000000,1.0,8.000000,342.000000,1.000000,1.44000


,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
0,Arbok,8.0,531.0,Hardy,1.0,Questions help guides,530.72,-0.28,Neutral,ArbokNeutral81,-,-,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.0,1.0,0.26,22.77,531.00,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
1,Arbok,9.0,574.0,Naughty,1.0,Rate My Mon,574.43,0.43,Naughty,ArbokNaughty91,Speed of Help,Main Skill Chance,3700.0,3276.72,Poison,Berries,Charge Energy S,14.0,32.0,40,Bean Sausage,103.0,1.09,0.90,1.00,1.0,0.8,0.26,22.77,538.37,ArbokNeutral91,80.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
2,Arbok,8.0,538.0,Quiet,1.0,Questions help guides,538.10,0.10,Quiet,ArbokQuiet81,Ingredient Finding,Exp Gains,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.00,1.00,1.2,1.0,0.26,22.77,530.61,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
3,Arcanine,5.0,958.0,Calm,3.0,,957.99,-0.01,Calm,ArcanineCalm53,Main Skill Chance,Speed of Help,2500.0,2728.00,Fire,Skills,Extra Helpful S,16.0,27.0,31,Fiery Herb,130.0,1.31,1.11,1.00,1.0,1.2,0.14,84.43,937.95,ArcanineNeutral53,31.0,130.0,1,True,,,,0.0,0.0,1.0,1.0
4,Arcanine,3.0,715.0,Lax,2.0,pokemon sleep general,715.12,0.12,Lax,ArcanineLax32,Energy Recovery,Main Skill Chance,2500.0,2490.00,Fire,Skills,Extra Helpful S,16.0,27.0,29,Fiery Herb,130.0,1.44,1.00,1.08,1.0,0.8,0.14,61.15,750.07,ArcanineNeutral32,29.0,130.0,1,True,,,,0.0,0.0,1.0,1.0


In [14]:
# Load previous fit

x0, unpack_info = pack(make_initial_guess())

hash_value = digest(data, x0)
filename = f"./results/least-squares-fit-{hash_value}.pickle"

opt = load(filename)
sol = unpack(opt.x, unpack_info)

opt

    message: `ftol` termination condition is satisfied.
    success: True
     status: 2
          x: [ 2.562e-01  2.545e-01 ...  2.195e-01  9.953e-01]
       cost: 970.1076591930191
       grad: [-1.729e-05  1.028e-04 ...  6.970e-05 -7.303e-04]
 optimality: 1.1403826778405346
       nfev: 50
       njev: 35

In [15]:
# Helpers

def truncated_normal_sample(size, mu, sigma, lower, upper):
    return stats.truncnorm.rvs( (lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma, size=size)

def round_noise(size):
    eps = 1e-6
    return truncated_normal_sample(size, mu=0.0, sigma=0.25, lower= -0.5+eps, upper= 0.5-eps)

def harmonic(a,b):
    return 2.0 / ( 1.0 / a + 1.0 / b )

In [16]:
# In the boostrap method,
# We create N synthetic datasets that are likely to represent the population
# Then we redo the fit on those, starting from the best fit in initial dataset.
# Finally, we collect statistics on the fitted variables between runs

# To create the synthetic datasets, we use sampling with replacement (allow duplicate)
# Because some Pokémon are rare, we use stratified sampling

stratas = data.groupby(['Pokemon'], group_keys=False)
target_group_size =  stratas.size().quantile(q=0.75)
    

n_boostraps = 50
all_opt_x = [opt.x]

In [17]:
cache_fit = True
boostrap_filename = f"./results/bootstrap-fit-{hash_value}.pickle"

if cache_fit and os.path.isfile(boostrap_filename):

    all_opt_x = load(boostrap_filename)
    print("Loaded from cache")

else:

    # WARNING: RUNNING THE OPTIMISATION N TIMES IS VERY LONG.
    #
    # YOU CAN INTERRUPT THE KERNEL (STOP BUTTON)
    # AND RUN THE NEXT CELLS IF YOU WANT TO SEE CURRENT PROGRESS
    #
    # THE COLLECTION `all_opt_x` IS DEFINED ABOVE THIS CELL
    # SO YOU CAN RESUME ADDING MORE RUNS

    while( len(all_opt_x) < n_boostraps ):

        print( "Boostrap run " + str(len(all_opt_x)) )

        # To create the synthetic datasets, we use sampling with replacement (allow duplicate)

        # We'll introduce a correction toward more equal sample size between Pokémon
        # We can motivate that by the fact stratified sampling should use population proportions
        # instead of current data proportions

        resampled = stratas.apply(lambda x: x.sample( round( harmonic(len(x),target_group_size) ), replace=True, ignore_index=True))

        # Add small noise to RP
        # We do so to simulate some unknown value that would round() to current RP

        referenceRP = resampled["RP"].to_numpy()
        referenceRP += round_noise(len(resampled))

        # Compute per sample information about help time, nature, sub-skills etc
        recomputed = make_precomputed_columns(resampled)

        # Put the pieces together

        def residual(x):
            return referenceRP - compute_rp(x, resampled, recomputed, unpack_info)

        # Redo the fit, starting from the optimal we found on current data
        opt2 = scipy.optimize.least_squares(residual, opt.x, loss="soft_l1", xtol=None, verbose=2)

        # Collect results for stats
        all_opt_x.append(opt2.x)

    # Save
    save(boostrap_filename, all_opt_x)

Boostrap run 1
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.0787e+03                                    1.32e+05    
       1              2         1.0318e+03      4.69e+01       2.04e+00       1.35e+06    
       2              3         1.0264e+03      5.42e+00       5.71e-01       2.08e+05    
       3              4         1.0260e+03      4.04e-01       1.38e-01       1.54e+03    
       4              5         1.0260e+03      2.44e-02       1.81e-02       1.98e+03    
       5              6         1.0260e+03      6.02e-04       1.81e-03       4.99e+01    
       6              7         1.0260e+03      8.24e-07       7.01e-05       1.26e+00    
`ftol` termination condition is satisfied.
Function evaluations 7, initial cost 1.0787e+03, final cost 1.0260e+03, first-order optimality 1.26e+00.
Boostrap run 2
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
   

In [29]:
# Stats
# We'll use the median as estimate for the mean.
# And 1.4826 * < median absolute deviation from the median > as estimate for std

center = np.median(all_opt_x, axis=0)

interval_std = 1.96 * np.std( all_opt_x, axis=0) # 95% confidence region
interval_mad = 3 * np.median( list(map(lambda x: np.abs(center-x), all_opt_x)), axis=0) # 95% confidence region

sol_center = unpack(center, unpack_info)
sol_interval_std = unpack(interval_std, unpack_info)
sol_interval_mad = unpack(interval_mad, unpack_info)

# Pretty display
pd.DataFrame({
    
    "Pokemon":game.pokedex.data["Pokemon"], 
    
    "ing%": sol_center["Pokemons ing fractions"]*100.0,
    #"conf (ing)": sol_interval_std["Pokemons ing fractions"]*100.0,
    "conf (ing)*": sol_interval_mad["Pokemons ing fractions"]*100.0,

    "skill% * skillValue": sol_center["Pokemons skill products"], 
    #"conf (skill)": sol_interval_std["Pokemons skill products"],
    "conf (skill)*": sol_interval_mad["Pokemons skill products"],


}).set_index("Pokemon")

,ing%,conf (ing)*,skill% * skillValue,conf (skill)*
Pokemon,,,,
Bulbasaur,25.611554,0.074337,16.734098,0.097596
Ivysaur,25.441933,0.039630,16.701053,0.027626
Venusaur,26.559776,0.072258,18.431858,0.046693
Charmander,20.035711,0.044472,9.682035,0.057468
Charmeleon,22.654273,0.039163,14.047527,0.044475
Charizard,22.381044,0.051165,14.030546,0.054302
Squirtle,26.995602,0.037574,17.628754,0.064462
Wartortle,27.046690,0.037718,17.569037,0.027129
Blastoise,27.433551,0.053064,18.425233,0.034736


In [ ]:
# Explain what we did with the re-sampling target size
# pd.DataFrame({'before': stratas.size(), 'after': stratas.apply(lambda x: round( harmonic(len(x), target_group_size) )) })

In [21]:
data[data["Pokemon"]=="Slaking"]

,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
2077,Slaking,27.0,2648.0,Calm,6.0,,2645.48,-2.52,Calm,SlakingCalm276Skill Level Up MEnergy Recovery ...,Main Skill Chance,Speed of Help,3800.0,3962.64,Normal,Berries,Ingredient Magnet S,12.0,28.0,54,Snoozy Tomato,110.0,0.90,1.1,1.00,1.0,1.2,0.35,304.27,2545.26,SlakingNeutral276Skill Level Up MEnergy Recove...,108.0,110.0,1,True,Skill Level Up M,Energy Recovery Bonus,,0.0,0.0,1.0,1.22
2078,Slaking,27.0,1615.0,Gentle,3.0,,1615.02,0.02,Gentle,SlakingGentle273Berry Finding SHelping bonus,Main Skill Chance,Energy Recovery,3800.0,3602.40,Normal,Berries,Ingredient Magnet S,12.0,28.0,54,Snoozy Tomato,110.0,0.99,1.0,0.92,1.0,1.2,0.35,115.55,1615.98,SlakingNeutral273Berry Finding SHelping bonus,162.0,110.0,1,True,Berry Finding S,Helping bonus,,0.0,0.0,1.0,1.22
2079,Slaking,27.0,2637.0,Quiet,6.0,,2637.27,0.27,Quiet,SlakingQuiet276Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3602.40,Normal,Berries,Ingredient Magnet S,12.0,28.0,54,Snoozy Tomato,110.0,0.99,1.0,1.00,1.2,1.0,0.35,413.80,2627.95,SlakingNeutral276Skill Level Up MSkill Trigger M,108.0,110.0,1,True,Skill Level Up M,Skill Trigger M,,0.0,0.0,1.0,1.00
2080,Slaking,29.0,2681.0,Quiet,6.0,,2681.82,0.82,Quiet,SlakingQuiet296Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3587.20,Normal,Berries,Ingredient Magnet S,12.0,28.0,56,Snoozy Tomato,110.0,1.00,1.0,1.00,1.2,1.0,0.35,413.80,2672.20,SlakingNeutral296Skill Level Up MSkill Trigger M,112.0,110.0,1,True,Skill Level Up M,Skill Trigger M,,0.0,0.0,1.0,1.00
2081,Slaking,30.0,2831.0,Quiet,6.0,,2836.48,5.48,Quiet,SlakingQuiet306Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3579.60,Normal,Berries,Ingredient Magnet S,12.0,28.0,57,Snoozy Tomato,110.0,1.00,1.0,1.00,1.2,1.0,0.35,413.80,2798.15,SlakingNeutral306Skill Level Up MSkill Trigger M,114.0,165.0,1,True,Skill Level Up M,Skill Trigger M,Snoozy Tomato,2.0,110.0,1.0,1.00
2082,Slaking,32.0,2859.0,Quiet,6.0,,2864.14,5.14,Quiet,SlakingQuiet326Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3564.40,Normal,Berries,Ingredient Magnet S,12.0,28.0,60,Snoozy Tomato,110.0,1.00,1.0,1.00,1.2,1.0,0.35,413.80,2828.13,SlakingNeutral326Skill Level Up MSkill Trigger M,120.0,165.0,1,True,Skill Level Up M,Skill Trigger M,Snoozy Tomato,2.0,110.0,1.0,1.00
2083,Slaking,33.0,2905.0,Quiet,6.0,,2909.88,4.88,Quiet,SlakingQuiet336Skill Level Up MSkill Trigger M,Ingredient Finding,Exp Gains,3800.0,3556.80,Normal,Berries,Ingredient Magnet S,12.0,28.0,62,Snoozy Tomato,110.0,1.01,1.0,1.00,1.2,1.0,0.35,413.80,2875.18,SlakingNeutral336Skill Level Up MSkill Trigger M,124.0,165.0,1,True,Skill Level Up M,Skill Trigger M,Snoozy Tomato,2.0,110.0,1.0,1.00


In [22]:
data[data["Pokemon"]=="Glaceon"]


,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
1153,Glaceon,9.0,773.0,Lonely,2.0,pokemon sleep general,773.23,0.23,Lonely,GlaceonLonely92,Speed of Help,Energy Recovery,3200.0,2833.92,Ice,Skills,Cooking Power-Up S,12.0,32.0,40,Moomoo Milk,98.0,1.27,0.9,0.92,1.0,1.0,0.21,79.15,754.21,GlaceonNeutral92,40.0,98.0,1,True,,,,0.0,0.0,1.0,1.00
1154,Glaceon,9.0,815.0,Relaxed,2.0,,814.79,-0.21,Relaxed,GlaceonRelaxed92,Energy Recovery,EXP Gains,3200.0,3148.80,Ice,Skills,Cooking Power-Up S,12.0,32.0,40,Moomoo Milk,98.0,1.14,1.0,1.08,1.0,1.0,0.21,79.15,754.63,GlaceonNeutral92,40.0,98.0,1,True,,,,0.0,0.0,1.0,1.00
1163,Glaceon,14.0,891.0,Bashful,2.0,,891.00,0.00,Neutral,GlaceonNeutral142Inventory Up M,-,-,3200.0,3116.80,Ice,Skills,Cooking Power-Up S,12.0,32.0,45,Moomoo Milk,98.0,1.15,1.0,1.00,1.0,1.0,0.21,79.15,891.00,GlaceonNeutral142Inventory Up M,45.0,98.0,1,True,Inventory Up M,,,0.0,0.0,1.0,1.13
